<font size="6"><b>RECURSIVE PARTIONING TREE TOY EXAMPLE</b></font>

In [ ]:
library(tidyverse)
library(data.table)
library(MASS) # for generating random samples from multivariate normal distribution
library(trialr) # for generating random correlation matrix from LKJ distribution
library(rpart) # for recursive partioning trees
library(visNetwork) # for better plotting recursive partioning trees

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=15) # for limiting the number of top and bottom rows of tables printed 

We will simulate a toy dataset to demonstrate decision tree algorithm for classification:

We will sample continous values from multivariate normal distribution and then discretize them to get factor variables:

Let's first create a random correlation matrix using the relevant LKJ distribution:

In [ ]:
set.seed(1)
matcor <- rlkjcorr(1, 3, 0.1)

In [ ]:
matcor

And let's sample some correlated random values from multivariate normal distribution:

In [ ]:
set.seed(1)
vals <- mvrnorm(2e2, rep(0, 3), matcor)

In [ ]:
vals

Check the correlation matrix of sampled values:

In [ ]:
cor(vals)

Similar to the generating correlation matrix we passed

Make it a data.table:

In [ ]:
vals_dt <- as.data.table(vals)

First column will be the response variable, others are independent variables:

In [ ]:
setnames(vals_dt, c("dep", "ind1", "ind2"))

Discretize values:

In [ ]:
vals_dt <- vals_dt %>% mutate_all(cut, c(-Inf, 0, Inf), c("no", "yes"))

Check the correlation among classes:

In [ ]:
cor(vals_dt %>% mutate_all(as.integer))

Let's visualize the possible splits:

In [ ]:
vals_dt %>%
mutate_at(c("ind1", "ind2"), as.integer) %>%
ggplot(aes(x = ind1, y = ind2, col = dep)) +
geom_jitter() +
geom_hline(yintercept = 1.5) +
geom_vline(xintercept = 1.5)

Let's formalize this through entropy measure:

$${\displaystyle \mathrm {H} (X):=-\sum _{x\in {\mathcal {X}}}p(x)\log p(x),}$$

(https://en.wikipedia.org/wiki/Entropy_(information_theory))

In [ ]:
entrop <- function(x)
{
    props <- prop.table(table(as.character(x)))
    sum(-props * log2(props))
}

And the gini impurity measure:

$${\displaystyle \operatorname {I} _{G}(p)=1-\sum _{i=1}^{J}p_{i}^{2}.}$$

(https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity)

In [ ]:
ginix <- function(x)
{
    props <- prop.table(table(as.character(x)))
    1- sum(props^2)
}

Select the independent variables to split across:

In [ ]:
vars <- c("ind1", "ind2")

See the weighted entropies across two variables:

In [ ]:
ents <- sapply(vars, function(x) vals_dt[, .(N = .N, en = entrop(dep)), by = get(x)][, sum(N * en)/sum(N)])

In [ ]:
ents

Recalculate using gini impurity values:

In [ ]:
ginis <- sapply(vars, function(x) vals_dt[, .(N = .N, en = ginix(dep)), by = get(x)][, sum(N * en)/sum(N)])

In [ ]:
ginis

They are parallel

Select the variable that cause the lower entropy:

In [ ]:
splitvar1 <- names(ents[which.min(ents)])
splitvar1

Split the data.table across this variable:

In [ ]:
vals_dt_l1 <- split(vals_dt, f = vals_dt[, .(get(splitvar1))])[c("no", "yes")]

And repeat the entropy calculation for both splits across the other variable (there is only one variable left, anyway, nothing to compare):

In [ ]:
lapply(vals_dt_l1, function(y)
    {
    sapply(setdiff(vars, splitvar1), function(x) y[, .(N = .N, en = entrop(dep)), by = get(x)][, sum(N * en)/sum(N)])
    }
)

Now let's see the information gain, the reduction in entropy, at the beginning and after each split:

First at the root:

In [ ]:
counts_dt0 <- vals_dt[, .N, by = c("dep")][, prop := N / sum(N)][]

In [ ]:
counts_dt0

Get the entropy:

In [ ]:
ent0 <- counts_dt0[, sum(-setdiff(prop, 0) * log2(setdiff(prop, 0)))]
ent0

And the error rate:

In [ ]:
er0 <- counts_dt0[, sum((N != max(N))*N) / sum(N)]
er0

Now, after the first split:

In [ ]:
counts_dt1 <- vals_dt[, .N, by = c("dep", splitvar1)][, prop := N / sum(N), by = splitvar1][]

In [ ]:
counts_dt1

The entropy value:

In [ ]:
ent1 <- counts_dt1[, .(N = sum(N), ent = sum(-setdiff(prop, 0) * log2(setdiff(prop, 0)))), by = splitvar1][, sum(N * ent) / sum(N)]
ent1

Entropy is reduced by:

In [ ]:
ent0 - ent1

The error rate:

In [ ]:
er1 <- counts_dt1[, sum((dep != ind1) * N) / sum(N)]
er1

And relative decrease in error:

In [ ]:
1 - er1 / er0

Keep that value in mind!

Now the second split:

In [ ]:
counts_dt2 <- vals_dt[, .N, by = c("dep", vars)][, prop := N / sum(N), by = vars][]

In [ ]:
counts_dt2

Let's look at the error rate for each split:

When ind1 == "yes" condition is not split further: (labels for the splitting variable are determined such that classification error is minimized)

In [ ]:
min(
counts_dt2[ind1 == "yes"][, sum((dep == ind1) * N) / sum(N)],
counts_dt2[ind1 == "yes"][, sum((dep != ind1) * N) / sum(N)])

And when the node is split further by ind2:

In [ ]:
min(
counts_dt2[ind1 == "yes"][, sum((dep == ind2) * N) / sum(N)],
counts_dt2[ind1 == "yes"][, sum((dep != ind2) * N) / sum(N)])

So the error rate increases with further split on the second variable for the ind1 == "yes" split

Now let's repeat it for ind1 == "no" split

In [ ]:
min(
counts_dt2[ind1 == "no"][, sum((dep == ind1) * N) / sum(N)],
counts_dt2[ind1 == "no"][, sum((dep != ind1) * N) / sum(N)])

And when the node is split further by ind2:

In [ ]:
min(
counts_dt2[ind1 == "no"][, sum((dep == ind2) * N) / sum(N)],
counts_dt2[ind1 == "no"][, sum((dep != ind2) * N) / sum(N)])

The error rate decreases for that split

Let's delete the second level split on the ind1 == "yes" node:

In [ ]:
counts_dt2b <- copy(counts_dt2)

In [ ]:
counts_dt2b[ind1 == "yes", ind2 := NA]

In [ ]:
counts_dt2b <- counts_dt2b[, .(N = sum(N)), by = c("dep", "ind1", "ind2")][, prop := N / sum(N), by = vars][]

In [ ]:
counts_dt2b

Calculate the entropy:

In [ ]:
ent2b <- counts_dt2b[, .(N = sum(N), ent = sum(-setdiff(prop, 0) * log2(setdiff(prop, 0)))), by = vars][, sum(N * ent) / sum(N)]
ent2b

Entropy is reduced by:

In [ ]:
ent1 - ent2b

So for short, **as long as the relative classification error decreases sufficiently**, at each node, next variable for split is chosen so that entropy is reduced most

Now let's make rpart do the heavy lifting:

In [ ]:
churn.rp <- rpart::rpart(dep ~ ., data = vals_dt)

How the splits are done:

In [ ]:
churn.rp

Summary of complexity parameters (CP) table:

In [ ]:
printcp(churn.rp)

Remember the first value in the CP column: The decrease in relative error we calculated before.

Relative error is the error at the depth level divided by the error at the root node (before any splits)

Let's extract the CP table

In [ ]:
cpdt <- churn.rp$cptable %>% as.data.table

In [ ]:
cpdt

CP is the change in relative error if further splits are made, divided by the increase in number of splits:

In [ ]:
cpdt[, -diff(`rel error`) / diff(nsplit)]

Let's visualize the tree:

In [ ]:
visNetwork::visTree(churn.rp)